In [1]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [4]:
file_path = '/home/fede/College.csv'
data = spark.read.csv(file_path, inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [10]:
output = assembler.transform(data)

In [13]:
from pyspark.ml.feature import StringIndexer

In [14]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [15]:
output_fixed = indexer.fit(output).transform(output)

In [16]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = true)



In [17]:
final_data = output_fixed.select('features','PrivateIndex')

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [24]:
from pyspark.ml.classification import (DecisionTreeClassifier, 
                                       RandomForestClassifier,
                                       GBTClassifier)

In [25]:
from pyspark.ml import Pipeline

In [47]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(numTrees=150,labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [48]:
dtc_model=dtc.fit(train_data)
rfc_model=rfc.fit(train_data)
gbt_model=gbt.fit(train_data)

In [49]:
dtc_pred = dtc_model.transform(test_data)
rfc_pred = rfc_model.transform(test_data)
gbt_pred = gbt_model.transform(test_data)

In [50]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [51]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [52]:
print('DTC')
print(my_binary_eval.evaluate(dtc_pred))

DTC
0.9077205224548219


In [53]:
print('RFC')
print(my_binary_eval.evaluate(rfc_pred))

RFC
0.9890857040615495


In [43]:
gbt_pred.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = true)
 |-- prediction: double (nullable = true)



In [44]:
rfc_pred.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [45]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',
                                               rawPredictionCol='prediction')

In [46]:
print('GBT')
print(my_binary_eval2.evaluate(gbt_pred))

GBT
0.9111647879763821


In [54]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [55]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', 
                                             metricName='accuracy')

In [56]:
rfc_acc = acc_eval.evaluate(rfc_pred)

In [57]:
rfc_acc

0.948051948051948